In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()                       # INPUT        - OUTPUT       - RECEPTIVE FIELD     
        self.conv1 = nn.Conv2d(1, 8, 3)                   # 28*28*1      - 26*26*8      - 3*3 
        self.bn1   = nn.BatchNorm2d(8)                        
        self.dp1   = nn.Dropout(p=0.10) 
        
        self.conv2 = nn.Conv2d(8, 16, 3 )                 # 26*26*8      - 24*24*16     - 5*5
        self.bn2   = nn.BatchNorm2d(16)                        
        self.dp2   = nn.Dropout(p=0.10)
        
        self.conv3 = nn.Conv2d(16, 16, 3 )                # 24*24*16     - 22*22*16     - 7*7
        self.bn3   = nn.BatchNorm2d(16)                        
        self.dp3   = nn.Dropout(p=0.10)

        self.pool1 = nn.MaxPool2d(2, 2)                   # 22*22*16     - 11*11*16     - 14*14
        self.bn4   = nn.BatchNorm2d(16)                        
        self.dp4   = nn.Dropout(p=0.10)

        self.dj1   = nn.Conv2d(16, 8, 1)                  # 11*11*16     - 11*11*8      - 14*14
        self.bn5   = nn.BatchNorm2d(8)                        
        self.dp5   = nn.Dropout(p=0.10)               

        self.conv4 = nn.Conv2d(8, 16, 3)                  # 11*11*8      - 9*9*16       - 16*16 
        self.bn6   = nn.BatchNorm2d(16)     
        self.dp6   = nn.Dropout(p=0.10)          

        self.conv5 = nn.Conv2d(16, 32, 3)                 # 9*9*16       - 7*7*32       - 18*18 
        self.bn7   = nn.BatchNorm2d(32)     
        self.dp7   = nn.Dropout(p=0.10)                       

        self.gap1  = nn.AvgPool2d(7,7)                    # 7*7*32       - 1*1*32       
        self.conv6 = nn.Conv2d(32,10,1)                   # 1*1*32       - 1*1*10       - 22*22        
        



    def forward(self, x):
        x = self.dp4(self.bn4(self.pool1(self.dp3(self.bn3(F.relu(self.conv3(self.dp2(self.bn2(F.relu(self.conv2(self.dp1(self.bn1(F.relu(self.conv1(x)))))))))))))))
        x = self.dp7(self.bn7(F.relu(self.conv5(self.dp6(self.bn6(F.relu(self.conv4(self.dp5(self.bn5(F.relu(self.dj1(x))))))))))))
        x = self.conv6(self.gap1(x))
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,168
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           2,320
       BatchNorm2d-8           [-1, 16, 22, 22]              32
           Dropout-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
      BatchNorm2d-11           [-1, 16, 11, 11]              32
          Dropout-12           [-1, 16, 11, 11]               0
           Conv2d-13            [-1, 8, 11, 11]             136
      BatchNorm2d-14            [-1, 8,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:

torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return correct

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

for epoch in range(1, 20):
  print(f'Epoch: {epoch}')
  train(model, device, train_loader, optimizer, epoch)
  result = test(model, device, test_loader)
  if result > 9940:
    break

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0611, Accuracy: 9818/10000 (98.18%)

Epoch: 2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0237, Accuracy: 9928/10000 (99.28%)

Epoch: 16


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0230, Accuracy: 9920/10000 (99.20%)

Epoch: 17


loss=0.0015242932131513953 batch_id=937: 100%|██████████| 938/938 [00:31<00:00, 30.23it/s]



Test set: Average loss: 0.0186, Accuracy: 9941/10000 (99.41%)

